<a href="https://colab.research.google.com/github/mostly-sunny/digital-health-hackathon/blob/main/Geonwoo's%20version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pycox - CoxPH Model
- Network : Test with sets, and find best network and lr by lowest brier score
- Input Variables : G1 ~ G300, Var1 ~ Var10, Treatment
- Output Variables : time, event
- Scaler : MinMaxScaler -> Var1 ~ Var10



In [31]:
pip install pycox

In [32]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper
import pandas as pd

import torch
import torchtuples as tt

from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [33]:
np.random.seed(123456)
_ = torch.manual_seed(123456)

- all-in-one.csv 파일은 유전자 변이 유무, 임상 변수, 생존 기간, 사망 여부, 치료 유무가 열로 존재하는 파일
- test-data-treat-and-untreat.csv 파일은 all-in-one의 열은 같은 602개의 데이터.
  - (0번째 행) : 유전자 변이 모두 0, 치료 0
  - (1번째 행) : 유전자 변이 모두 0, 치료 1
  - (2~301번째 행) : 유전자 변이 n-1에만 1, 치료 0
  - (302~601번재 행) : 유전자 변이 n-301에만 1, 치료 1
- pandas 라이브러리에 있는 csv 파일을 DataFrame으로 바꾸어주는 read_csv 함수를 이용하여 파일을 읽어 들임.
- DataFrame은 표를 나타내는 데이터 타입임.

In [34]:
dataset = pd.read_csv('/content/all-in-one.csv')
dataset_for_hr = pd.read_csv('/content/test-data-treat-and-untreat.csv')

- 위에서 읽어들인 dataset 중에서 20%는 검증(_val -> validation)을 위해 sampling 한다.
- 남은 80%의 데이터에서도 20%는 테스트(_test)를 위해 sampling 한다.

In [35]:
dataset_val = dataset.sample(frac=0.3)
dataset_train = dataset.drop(dataset_val.index)
dataset_test = dataset_train.sample(frac=0.3)
dataset_train = dataset_train.drop(dataset_test.index)

- columns_standardize : 임상변수 - 0~9사이의 값을 표준화
- columns_leave : 유전자 변이 유무 + 치료 유무 - 0과 1로 표현돼 있기 때문에 표준화 필요 없음.
- DataFrameMapper는 pandas DataFrame에서 원하는 열을 뽑아서 리스트로 만들어줌.
- 리스트로 만들때 StandardScaler()가 포함된 열은 표준화를 시킨 뒤, 그리고 None이면 갖고 있는 값을 그대로 넣음.

In [36]:
columns_standardize = ['Var' + str(i) for i in range(1,11)]
columns_leave = ['G' + str(i) for i in range(1,301)]
columns_leave += ['Treatment']

# standardize = [([col], StandardScaler()) for col in columns_standardize]
standardize = [([col], MinMaxScaler()) for col in columns_standardize]

leave = [(col, None) for col in columns_leave]

x_mapper = DataFrameMapper(leave + standardize)

- 위에서 만든 DataFrameMapper로 DataFrame 중 x(입력) 데이터를 모델이 학습할 수 있게끔 리스트 형식으로 바꾸어 준다.



In [37]:
x_train = x_mapper.fit_transform(dataset_train).astype('float32')
x_val = x_mapper.transform(dataset_val).astype('float32')
x_test = x_mapper.transform(dataset_test).astype('float32')
x_for_hr = x_mapper.transform(dataset_for_hr).astype('float32')

- DataFrame (표)에서 Y(출력)데이터인 time(생존기간)과 event(사망여부)를 뽑아 출력 데이터를 추린다.
- 검증(Validation)을 위한 입력-출력 세트 val을 만든다.

In [38]:
get_target = lambda df: (df['time'].values, df['event'].values)
y_train = get_target(dataset_train)
y_val = get_target(dataset_val)

durations_test, events_test = get_target(dataset_test)
val = x_val, y_val

함수 make_net : network을 생성해 리턴하는 함수
- input과 output의 노드 수, 은닉층 수, 은닉층의 노드 수 설정 가능

In [39]:
def make_net(in_features, out_features, hidden, nodes):
  if hidden == 1:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 2:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 3:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 4:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
      
      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, out_features)
    )
  return network

- in_features : 입력데이터의 개수 (x_train.shape : 311 = 300(유전자) + 10(임상변수) + 1(치료유무))
- out_features : 출력노드의 개수

- hidden_layers : 은닉층 수를 가지고 있는 리스트
- number_nodes : 은닉층에 있는 노드 수를 가지고 있는 리스트
- learning_rates : 테스트할 학습률을 가지고 있는 리스트
- brier_scores = brier score을 계산해 append

In [40]:
in_features = x_train.shape[1]
out_features = 1

hidden_layers = [2]
number_nodes = [2060]
learning_rates = [0.0001, 0.001, 0]
brier_scores = []

for i in hidden_layers:
  for j in number_nodes:
    for k in learning_rates:
      net = make_net(in_features, out_features, i, j)
      model = CoxPH(net, tt.optim.Adam)
      batch_size = 639

      if k == 0:
        lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance = 10)
        model.optimizer.set_lr(lrfinder.get_best_lr())
      else:
        model.optimizer.set_lr(k)
      
      epochs = 512
      callbacks = [tt.callbacks.EarlyStopping()]
      verbose = True

      %%time
      model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
      _ = model.compute_baseline_hazards()
      surv = model.predict_surv_df(x_test)
      
      # calculate ratio
      log_partial_hazard = model.predict(x_for_hr)
      partial_hazard = [np.exp(lph) for lph in log_partial_hazard]

      treat_hr = []
      # ratio with treated and untreated
      for g in range(300):
        treat_hr.append([partial_hazard[g+302]/partial_hazard[g+2],'G' + str(g+1)])
      treat_hr.sort()

      # evaluation
      ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
      time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
      score = ev.integrated_brier_score(time_grid)

      if k == 0:
        brier_scores.append([score, i, j, lrfinder.get_best_lr(), treat_hr[:10]])
      else:
        brier_scores.append([score, i, j, k, treat_hr[:10]])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
0:	[0s / 0s],		train_loss: 5.3550,	val_loss: 4.6684
1:	[0s / 1s],		train_loss: 5.0566,	val_loss: 4.6657
2:	[0s / 1s],		train_loss: 4.7983,	val_loss: 4.6624
3:	[0s / 2s],		train_loss: 4.6320,	val_loss: 4.6588
4:	[0s / 2s],		train_loss: 4.4746,	val_loss: 4.6547
5:	[0s / 3s],		train_loss: 4.3693,	val_loss: 4.6503
6:	[0s / 3s],		train_loss: 4.2332,	val_loss: 4.6452
7:	[0s / 4s],		train_loss: 4.1383,	val_loss: 4.6397
8:	[0s / 5s],		train_loss: 4.0528,	val_loss: 4.6336
9:	[0s / 5s],		train_loss: 3.9636,	val_loss: 4.6268
10:	[0s / 6s],		train_loss: 3.8791,	val_loss: 4.6196
11:	[0s / 6s],		train_loss: 3.8200,	val_loss: 4.6115
12:	[0s / 7s],		train_loss: 3.7611,	val_loss: 4.6028
13:	[0s / 7s],		train_loss: 3.6951,	val_loss: 4.5940
14:	[0s / 8s],		train_loss: 3.6472,	val_loss: 4.5852
15:	[0s / 8s],		train_loss: 3.6042,	val_loss: 4.5761
16:	[0s / 9s],		train_loss: 3.5781,	val_loss: 4.5670
17:	[0s / 10s],		train_loss: 3.5598,	val_loss:

- brier_score가 가장 작은 것부터 정렬


In [41]:
brier_scores.sort()
selected_genes = []
for i in range(10):
  selected_genes.append(brier_scores[0][4][i][1])

- brier_score가 가장 좋은 케이스 출력

In [42]:
gene_count= {}
for i in brier_scores:
  lst = []
  for j in i[4]:
    lst.append(j[1])
  print(lst)
  for k in lst:
    if k in gene_count.keys():
      gene_count[k] += 1
    else:
      gene_count[k] = 1
sorted(gene_count.items(), key=lambda x: x[1], reverse=True)

['G35', 'G131', 'G161', 'G264', 'G259', 'G124', 'G25', 'G47', 'G214', 'G204']
['G69', 'G180', 'G198', 'G96', 'G29', 'G233', 'G202', 'G127', 'G265', 'G103']
['G35', 'G47', 'G74', 'G16', 'G171', 'G168', 'G34', 'G135', 'G204', 'G186']


[('G35', 2),
 ('G47', 2),
 ('G204', 2),
 ('G131', 1),
 ('G161', 1),
 ('G264', 1),
 ('G259', 1),
 ('G124', 1),
 ('G25', 1),
 ('G214', 1),
 ('G69', 1),
 ('G180', 1),
 ('G198', 1),
 ('G96', 1),
 ('G29', 1),
 ('G233', 1),
 ('G202', 1),
 ('G127', 1),
 ('G265', 1),
 ('G103', 1),
 ('G74', 1),
 ('G16', 1),
 ('G171', 1),
 ('G168', 1),
 ('G34', 1),
 ('G135', 1),
 ('G186', 1)]

In [43]:
print("Brier Score :", brier_scores[0][0])
print("Hidden Layers :", brier_scores[0][1])
print("Number of Nodes :", brier_scores[0][2])
print("Learning Rate :", brier_scores[0][3])
print("Selection :", selected_genes)

Brier Score : 0.0499072177206306
Hidden Layers : 2
Number of Nodes : 2060
Learning Rate : 0.001484968262254472
Selection : ['G35', 'G131', 'G161', 'G264', 'G259', 'G124', 'G25', 'G47', 'G214', 'G204']


NAN 값을 기준으로 정렬이 끊어진 문제 발견

In [44]:
brier_scores

[[0.0499072177206306,
  2,
  2060,
  0.001484968262254472,
  [[array([0.601699], dtype=float32), 'G35'],
   [array([0.60666573], dtype=float32), 'G131'],
   [array([0.60804623], dtype=float32), 'G161'],
   [array([0.60933214], dtype=float32), 'G264'],
   [array([0.6115791], dtype=float32), 'G259'],
   [array([0.61353236], dtype=float32), 'G124'],
   [array([0.6161169], dtype=float32), 'G25'],
   [array([0.6171879], dtype=float32), 'G47'],
   [array([0.6200422], dtype=float32), 'G214'],
   [array([0.6205481], dtype=float32), 'G204']]],
 [0.050703768027525804,
  2,
  2060,
  0.001,
  [[array([0.5437677], dtype=float32), 'G69'],
   [array([0.54808986], dtype=float32), 'G180'],
   [array([0.5550878], dtype=float32), 'G198'],
   [array([0.5714226], dtype=float32), 'G96'],
   [array([0.57293653], dtype=float32), 'G29'],
   [array([0.57497305], dtype=float32), 'G233'],
   [array([0.5756375], dtype=float32), 'G202'],
   [array([0.57575196], dtype=float32), 'G127'],
   [array([0.57608366], dtyp

In [45]:
print("brier_score, hidden layer, number of nodes, learning rate")
for i in brier_scores:
  print(i[:-1])

brier_score, hidden layer, number of nodes, learning rate
[0.0499072177206306, 2, 2060, 0.001484968262254472]
[0.050703768027525804, 2, 2060, 0.001]
[0.051220967069960065, 2, 2060, 0.0001]
